This presents the eploration of the superstore sales data in SQL. We will attempt check if there is any data cleaning neccessary

Source of data is kaggle: [https://www.kaggle.com/blurredmachine/superstore-time-series-dataset](https://www.kaggle.com/blurredmachine/superstore-time-series-dataset)

Jupyter Notebook

I am only displaying top 5 rows in each table for easier viewing in this notebook  

Key tasks inspired by projectpro.io

- Use the LEAD window function to create a new column sales\_next that displays the sales of the next row in the dataset. This function will help you quickly compare a given row’s values and values in the next row.
    
- Create a new column sales\_previous to display the values of the row above a given row.
    
- Rank the data based on sales in descending order using the RANK function.
    
- Use common SQL commands and aggregate functions to show the monthly and daily sales, discount, and quantity averages.
    
- Evaluate moving averages using the window functions

1\. let's just explore the data by looking at all the variables

In [3]:
select top (5) *
from superstore

(5 rows affected)

Total execution time: 00:00:00.189

Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales,Quantity,Discount
8002,CA-2015-110870,2015-12-12,2015-12-15,First Class,KD-16270,Karen Daniels,Consumer,United States,Los Angeles,California,90032,West,TEC-AC-10002926,Technology,Accessories,Logitech Wireless Marathon Mouse M705,299.94,6,0
8003,CA-2015-110870,2015-12-12,2015-12-15,First Class,KD-16270,Karen Daniels,Consumer,United States,Los Angeles,California,90032,West,OFF-SU-10001225,Office Supplies,Supplies,Staple remover,25.76,7,0
8004,CA-2014-143210,2014-12-01,2014-12-03,First Class,AA-10645,Anna Andreadi,Consumer,United States,Lowell,Massachusetts,1852,East,TEC-PH-10004434,Technology,Phones,Cisco IP Phone 7961G VoIP phone - Dark gray,271.9,2,0
8005,CA-2014-143210,2014-12-01,2014-12-03,First Class,AA-10645,Anna Andreadi,Consumer,United States,Lowell,Massachusetts,1852,East,FUR-FU-10003878,Furniture,Furnishings,"Linden 10"" Round Wall Clock, Black",45.839999999999996,3,0
8006,CA-2014-143210,2014-12-01,2014-12-03,First Class,AA-10645,Anna Andreadi,Consumer,United States,Lowell,Massachusetts,1852,East,FUR-FU-10002268,Furniture,Furnishings,Ultra Door Push Plate,9.82,2,0


2\. Let's check to ensure there is no null data in sales, discount and quantity that we plan to aggregate.

There is no null data

In [4]:
select top (5) sales,discount,quantity
from superstore
where sales is null or discount is null or quantity is null

(0 rows affected)

Total execution time: 00:00:00.037

sales,discount,quantity


3. Do you we have any supsicion data in sales, quantity and discount? Let's create statistical description

In [5]:
select top (5) avg(sales) as AvgSales,avg(discount) as AvgDisc,avg(quantity) as AvgQty,
max(sales) as MaxSales,max(discount) as MaxDisc,avg(quantity) as MaxQty,
Min(sales) as MinSales,Min(discount) as MinDisc,Min(quantity) as MinQty
from superstore



(1 row affected)

Total execution time: 00:00:00.098

AvgSales,AvgDisc,AvgQty,MaxSales,MaxDisc,MaxQty,MinSales,MinDisc,MinQty
229.97954234821879,0.16968389362769473,3,13999.96,0.8,3,0.5559999999999998,0,1


Nothing suspicious

4\. Use the LEAD window function to create a new column sales\_next that displays the sales of the next row in the dataset. 

I decided to partition by product so that I can compare current and next sales of the same product. Clearly, you can see there are many product with limited sales data

In [6]:
select top (5) product_name, sales, LEAD(sales,1) over(partition by product_name order by product_name) as sales_next
from superstore
order by product_name asc

(5 rows affected)

Total execution time: 00:00:00.125

product_name,sales,sales_next
#6 3/4 Gummed Flap White Envelopes,15.840000000000002,NULL
"1/4 Fold Party Design Invitations & White Envelopes, 24 8-1/2"" X 11"" Cards, 25 Env./Pack",5.88,44.099999999999994
"#10- 4 1/8"" x 9 1/2"" Security-Tint Envelopes",22.919999999999998,NULL
"#10 White Business Envelopes,4 1/8 x 9 1/2",109.69,125.36
"#10 White Business Envelopes,4 1/8 x 9 1/2",125.36,NULL


5. Create a new column sales\_previous to display the values of the row above a given row using lag window funtion.

In [7]:
select top (5) product_name, sales, LAG(sales,1) over(partition by product_name order by product_name) as sales_next
from superstore
order by product_name asc

(5 rows affected)

Total execution time: 00:00:00.066

product_name,sales,sales_next
"#10- 4 1/8"" x 9 1/2"" Security-Tint Envelopes",22.919999999999998,NULL
"#10 White Business Envelopes,4 1/8 x 9 1/2",109.69,NULL
"#10 White Business Envelopes,4 1/8 x 9 1/2",125.36,109.69
#6 3/4 Gummed Flap White Envelopes,15.840000000000002,NULL
"1/4 Fold Party Design Invitations & White Envelopes, 24 8-1/2"" X 11"" Cards, 25 Env./Pack",5.88,NULL


6. Rank the data based on sales in descending order using the RANK functiion. In this example, I have decided to rank by category.

In [8]:
select top (5) category, product_name, sales, RANK() OVER (PARTITION BY category ORDER BY sales DESC) AS SalesRankCat
from superstore
order by category desc

(5 rows affected)

Total execution time: 00:00:00.059

category,product_name,sales,SalesRankCat
Technology,Canon imageCLASS 2200 Advanced Copier,13999.96,1
Technology,"HP Designjet T520 Inkjet Large Format Printer - 24"" Color",8749.95,2
Technology,Canon PC1060 Personal Laser Copier,4899.93,3
Technology,Zebra ZM400 Thermal Label Printer,4643.8,4
Technology,Okidata MB760 Printer,4476.8,5


7\. Rank the data based on sales in descending order using the RANK functiion. In this example, we will rank all the product in a single list

In [9]:
select top (5) product_name, sales, RANK() OVER (ORDER BY sales DESC) AS SalesRank
from superstore
order by sales desc

(5 rows affected)

Total execution time: 00:00:00.073

product_name,sales,SalesRank
Canon imageCLASS 2200 Advanced Copier,13999.96,1
GBC Ibimaster 500 Manual ProClick Binding System,9892.74,2
"HP Designjet T520 Inkjet Large Format Printer - 24"" Color",8749.95,3
Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind,5083.96,4
Canon PC1060 Personal Laser Copier,4899.93,5


8\. Use common SQL commands and aggregate functions to show the daily sales averages.

In [10]:
select top (5) order_date,avg(sales) as DailyAvgSales, avg(discount) as DailyAvgDiscount, avg(quantity) as DailyAvgQty
from superstore
group by order_date
order by order_date asc

(5 rows affected)

Total execution time: 00:00:00.037

order_date,DailyAvgSales,DailyAvgDiscount,DailyAvgQty
2014-01-13,20.423000000000002,0.44999999999999996,4
2014-01-21,25.248,0.2,3
2014-01-23,23.009999999999998,0,4
2014-01-26,76.32000000000001,0,2
2014-02-02,99.64800000000001,0.44999999999999996,3


9\. Use common SQL commands and aggregate functions to show the monthly sales averages.

In [11]:
-- create a temporary table
drop table if exists TempSales3
create table TempSales3(order_date date, monthly varchar(255), sales numeric, quantity numeric, discount numeric)
insert into TempSales3
--covert order date to MMMYYYY
Select order_date, replace(right(CONVERT(Varchar(255),order_date,106), 8), ' ', '-') as Monthly,sales, quantity, discount
from superstore
order by order_date asc

select top (5) convert(date,concat('01-',monthly),106) as Order_Date1,
avg(sales) as MonthlySales
, avg(quantity) as MonthlyQty
, avg(discount) as MonthlyDiscount 
from TempSales3
group by monthly 
order by Order_Date1


(1993 rows affected)

(5 rows affected)

Total execution time: 00:00:00.186

Order_Date1,MonthlySales,MonthlyQty,MonthlyDiscount
2014-01-01,37.714285,3.571428,0.142857
2014-02-01,79.000000,3.157894,0.105263
2014-03-01,176.111111,3.444444,0.027777
2014-04-01,231.346153,3.846153,0.000000
2014-05-01,258.875000,3.500000,0.125000


10. Rolling average and total of sales, discount and quantity

In [12]:
select top (5) order_date,
--average
avg(sales) over(order by order_date) as RollingAvgSales,
avg(quantity) over(order by order_date) as RollingAvgQty,
avg(discount) over(order by order_date) as RollingAvgDiscount,
--total
sum(sales) over(order by order_date) as RollingTotalSales,
sum(quantity) over(order by order_date) as RollingTotalQty
--sum(discount) over(order by order_date) as RollingTotalDiscount
from superstore
order by order_date asc

(5 rows affected)

Total execution time: 00:00:00.040

order_date,RollingAvgSales,RollingAvgQty,RollingAvgDiscount,RollingTotalSales,RollingTotalQty
2014-01-13,20.423000000000002,4,0.44999999999999996,40.846000000000004,9
2014-01-13,20.423000000000002,4,0.44999999999999996,40.846000000000004,9
2014-01-21,22.031333333333336,4,0.36666666666666664,66.09400000000001,12
2014-01-23,22.422800000000002,4,0.21999999999999997,112.114,21
2014-01-23,22.422800000000002,4,0.21999999999999997,112.114,21
